# Students
- GHAITH Sarahnour (M2QF & ENSIIE)
- ROISEUX Thomas (M2QF & ENSIIE)

# Introduction
## Context

The goal of this project is to study 
the temporal evolution of temperature and wind in France, across one year.

## Required packages
- `pandas` : to manipulate dataframes.
- `numpy` : to manipulate arrays.
- `matplotlib` : to plot graphs.
- `cartopy` : to plot maps.
- `IPython` : to display dataframes in Jupyter Notebook.
- `scikit-learn` : to use machine learning algorithms.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta

from IPython.display import display

## Data importation
### Preparing GPS dataframe

In [ ]:
gps_df = pd.read_csv("dataGPS.csv", header=None, sep=";")
gps_df.columns = ["Temp ID", "Lattitude", "Longitude"]
gps_df.insert(1, "Wind ID", gps_df["Temp ID"].str.replace("TEMP", "VVENT"))

display(gps_df.head())

### Preparing temperature dataframe and wind dataframe

In [ ]:
year = 2019
hours = [datetime(year, 1, 1, 0, 0, 0) + timedelta(hours=i) for i in range(8760)]

In [ ]:
temp_df = pd.read_csv("dataTemp.csv", header=None, sep=";", index_col=0)
temp_df.index.name = "Temperature ID"
temp_df.columns = hours


display(temp_df.head())


wind_df = pd.read_csv("dataWind.csv", header=None, sep=";", index_col=0)
wind_df.index.name = "Wind ID"
wind_df.columns = hours


display(wind_df.head())

## Example: weather in Paris
We are goinng to study the weather in Paris, the capital of France, as an example.
It is located at 48.51° N, 2.21° E.

Let's firstplace in on a map.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax.set_extent([-5, 9, 42, 52])
ax.set_title("France")
ax.stock_img()

x, y = 2.217999, 48.512381
ax.plot(x, y, "r*", markersize=15)
ax.text(x, y, "Paris")
plt.show()

Now, we are going to plot the evolution of temperature and wind in Paris, across one year.

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Temperature in Paris")
plt.plot(temp_df.columns, temp_df.iloc[33, :], color="blue")
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Wind in Paris")
plt.plot(wind_df.columns, wind_df.iloc[33, :], color="blue")
plt.show()

# Preliminaries